**Import required libraries and scripts**

In [ ]:
#Import required libraries and scripts
from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions import *
from scripts.clustering_functions import *
from scripts.rescoring_functions import *
from scripts.ranking_functions import *
from scripts.performance_calculation import *
from scripts.get_pocket import *
from scripts.dogsitescorer import *
import numpy as np
import os

software = '/home/tony/CADD22/software'
protein_file = '/home/tony/CADD22/wocondock_main/2o1x_A_apo_protoss.pdb'
ref_file = '/home/tony/CADD22/wocondock_main/2o1x_A_lig_protoss.sdf'
docking_library = '/home/tony/CADD22/wocondock_main/500_of_FCHGroup_LeadLike.sdf'
docking_programs = ['SMINA','GNINA','PLANTS']
id_column = 'ID'
n_poses = 10
exhaustiveness = 4

#Initialise variables and create a temporary folder
w_dir = os.path.dirname(protein_file)
print('The working directory has been set to:', w_dir)
create_temp_folder(w_dir+'/temp')

In [ ]:
#Import required libraries and scripts
from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions import *
from scripts.clustering_functions import *
from scripts.rescoring_functions import *
from scripts.ranking_functions import *
from scripts.performance_calculation import *
import numpy as np
import os

software = '/home/tony/CADD22/software'
protein_file = '/home/tony/CADD22/wocondock_test_1k/receptor_protoss_prepared.pdb'
ref_file = '/home/tony/CADD22/wocondock_test_1k/crystal_ligand_protoss.sdf'
docking_library = '/home/tony/CADD22/wocondock_test_1k/merged_actives_decoys.sdf'
docking_programs = ['SMINA','GNINA','PLANTS']
id_column = 'ID'
n_poses = 10
exhaustiveness = 4

#Initialise variables and create a temporary folder
w_dir = os.path.dirname(protein_file)
print('The working directory has been set to:', w_dir)
create_temp_folder(w_dir+'/temp')

In [1]:
#Import required libraries and scripts
from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions import *
from scripts.clustering_functions import *
from scripts.rescoring_functions import *
from scripts.ranking_functions import *
from scripts.performance_calculation import *
import numpy as np
import os

software = '/home/tony/CADD22/software'
protein_file = '/home/tony/CADD22/wocondock_performance_aldr/receptor_protoss_prepared.pdb'
ref_file = '/home/tony/CADD22/wocondock_performance_aldr/crystal_ligand_protoss.sdf'
docking_library = '/home/tony/CADD22/wocondock_performance_aldr/merged_actives_decoys.sdf'
docking_programs = ['PLANTS']
clustering_metrics = ['RMSD', 'spyRMSD', 'espsim', 'USRCAT', '3DScore', 'bestpose']
rescoring_functions = ['gnina', 'AD4', 'chemplp', 'rfscorevs']
id_column = 'ID'
n_poses = 10
exhaustiveness = 4

#Initialise variables and create a temporary folder
w_dir = os.path.dirname(protein_file)
print('The working directory has been set to:', w_dir)
create_temp_folder(w_dir+'/temp')

[15:23:37] Initializing Normalizer


The working directory has been set to: /home/tony/CADD22/wocondock_performance_aldr
The folder: /home/tony/CADD22/wocondock_performance_aldr/temp already exists


In [ ]:
pocket_definition = binding_site_coordinates_dogsitescorer(protein_file, w_dir, method='volume')

In [ ]:
cleaned_pkasolver_df = prepare_library(docking_library, id_column, software, 'pkasolver')

In [ ]:
all_poses = docking(protein_file, ref_file, software, docking_programs, exhaustiveness, n_poses)

In [ ]:
docking_splitted(w_dir, protein_file, ref_file, software, docking_programs, exhaustiveness, n_poses)

In [ ]:
all_poses = fetch_poses_splitted(w_dir, n_poses, '/home/tony/CADD22/wocondock_performance_aldr/temp/split_final_library')

In [ ]:
for metric in clustering_metrics:
    cluster_numpy_futures(f'{metric}', w_dir, protein_file)

**Rescoring**

The file containing all the cluster centers is then rescored using all scoring functions available (GNINA, Vina, AutoDock4, PLP, CHEMPLP, RF-Score-VS). The rescored output is return as a dataframe.

In [ ]:
for metric in clustering_metrics:
    print(metric)
    rescore_all(w_dir, protein_file, ref_file, software, w_dir+f'/temp/clustering/{metric}_clustered.sdf', rescoring_functions, 1)


**Final ranking methods**

This code calculates the final ranking of compounds using various methods.
*Method 1* : Calculates ECR value for each cluster center, then outputs the top ranked center.
*Method 2* : Calculates ECR value for each cluster center, then outputs the average ECR value for each compound.
*Method 3* : Calculates the average rank of each compound, then ouputs the corresponding ECR value for each compound.
*Method 6* : Calculates Z-score for each cluster center, then ouputs the top ranked center.
*Method 7* : Calculates Z-score for each cluster center, then ouputs the average Z-score for each compound.

All methods are then combined into a single dataframe for comparison purposes.

In [ ]:
apply_consensus_methods(w_dir, ['RMSD', 'espsim', 'spyRMSD', 'USRCAT', '3DScore', 'bestpose'])

In [ ]:
df1 = pd.read_csv('/home/tony/CADD22/wocondock_performance_aldr/temp/rescoring_3DScore_clustered/allposes_rescored.csv')
df2 = pd.read_csv('/home/tony/CADD22/wocondock_performance_aldr/temp/ranking/3DScore_standardised_ranked.csv')
df3 = pd.read_csv('/home/tony/CADD22/wocondock_performance_aldr/temp/ranking/3DScore_standardised.csv')
#df3 = df3.rename(columns={'AD4_Affinity':'AD4_Affinity_S', 'CHEMPLP':'CHEMPLP_S', 'GNINA_Affinity_S':'GNINA_Affinity_S', 'GNINA_CNN_Score':'GNINA_CNN_Score_S', 'GNINA_CNN_Affinity_S':'GNINA_CNN_Affinity_S', 'CNN_VS':'CNN_VS_S', 'RFScoreVS':'RFScoreVS_S'})
merged_df = pd.merge(df1, df2, on='Pose ID', how='outer')
merged_df = pd.merge(merged_df, df3, on='Pose ID', how='outer')
merged_df = merged_df[sorted(merged_df.columns)]
display(merged_df)

In [ ]:
show_correlation('/home/tony/CADD22/wocondock_main/temp/ranking/method_results.csv')

In [2]:
calculate_EFs(w_dir, docking_library)